In [1]:
import scrapy
from scrapy.crawler import CrawlerProcess
from scrapy import Request
import re
import pandas as pd

class ProductList(scrapy.Item):
    X1_product_code  =scrapy.Field()
    X2_product_category=scrapy.Field()
    X3_product_name = scrapy.Field()
    X4_product_rating= scrapy.Field()
    X5_product_oprice= scrapy.Field()
    X6_product_dprice= scrapy.Field()
    X7_website       = scrapy.Field()
    
#all_items
class go(scrapy.Spider):
    name='test'
    allowed_domains=['www.klook.com/zh-HK']
    start_urls=['https://www.klook.com/zh-HK/search/?template_id=3&start={}&type=country&container_type=country&revamp_template_id=3&revamp_city_id=105,57,59,58,288,60,61,62,161,174,177,179,182,184,187,256,257&revamp_country_id=20&revamp_index=4'.format(i+1)for i in range(3)]
    ROBOTTXT_OBEY=False

    def start_requests(self):
        headers= {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36'}
        for url in self.start_urls:
            yield Request(url, headers=headers)  
    
    def parse(self,response):
        item = ProductList()
        target = response.css("div.m_justify_list.m_radius_box.act_card.act_card_sm.a_sd_move.j_activity_item.js-item ")
        for tag in target:
            
            website=tag.css("a.j_activity_item_link::attr(href)")[0].extract()
            item["X1_product_code"]=website[16:].split("-")[0]
            item["X2_product_category"]="特色活動&體驗"
            
            try:
                product_name=tag.css("p.title::text")[0].extract()
                item["X3_product_name"]=product_name
            except:
                item["X3_product_name"]="NA"
            
            try:
                product_rating=tag.css("span.t14.star_score::text")[0].extract()
                item["X4_product_rating"]=product_rating
            except:
                item["X4_product_rating"]="NA"
            
            try:
                product_oprice=tag.css("p.del_price.u_t_gray_9 del::text")[0].extract().replace(" ","")
                item["X5_product_oprice"]=product_oprice
            except:
                item["X5_product_oprice"]="NA"
                
            try:
                product_dprice=tag.css("span.latest_price b::text")[0].extract().replace("HK$ ","").replace(" ","").replace("\n","")
                item["X6_product_dprice"]=product_dprice
            except:
                item["X6_product_dprice"]="NA"                            

            item["X7_website"]=("https://www.klook.com"+website)
            yield item

process = CrawlerProcess({
'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)',
'FEED_FORMAT': 'csv',
'FEED_URI': 'CHdata_Activities&Experiences.csv'
})
process.crawl(go)
process.start()

2019-10-17 14:25:11 [scrapy.utils.log] INFO: Scrapy 1.7.3 started (bot: scrapybot)
2019-10-17 14:25:11 [scrapy.utils.log] INFO: Versions: lxml 4.3.4.0, libxml2 2.9.9, cssselect 1.1.0, parsel 1.5.2, w3lib 1.21.0, Twisted 19.7.0, Python 3.7.3 (default, Apr 24 2019, 15:29:51) [MSC v.1915 64 bit (AMD64)], pyOpenSSL 19.0.0 (OpenSSL 1.1.1c  28 May 2019), cryptography 2.7, Platform Windows-10-10.0.17763-SP0
2019-10-17 14:25:11 [scrapy.crawler] INFO: Overridden settings: {'FEED_FORMAT': 'csv', 'FEED_URI': 'CHdata_Activities&Experiences.csv', 'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}
2019-10-17 14:25:11 [scrapy.extensions.telnet] INFO: Telnet Password: 33ba7909ccf98794
2019-10-17 14:25:11 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2019-10-17 14:25:11 [scrapy.middleware] INFO: Enabled downloader mid

In [2]:
import pandas as pd
df=pd.read_csv("CHdata_Activities&Experiences.csv")
print(len(df))

49
